<a href="https://colab.research.google.com/github/AbdirayimovS/OnDeepLearningEssentials/blob/main/On_Deep_Learning_Ian_Goodfellow_et_al_Naive_Bayes_from_scratch_%7C_Python3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ucimlrepo

In [ ]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
spambase = fetch_ucirepo(id=94)

# data (as pandas dataframes)
X = spambase.data.features
# X = X.iloc[:, :2]
y = spambase.data.targets

In [ ]:
X.head(3)

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,word_freq_conference,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.0,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.0,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.0,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259


In [ ]:
y = y['Class']

In [ ]:
X.isnull().sum().sum()

0

## Custom Class

Bayes' theorem is stated mathematically as the following equation:

![image.png](https://i.postimg.cc/FKrWdpxd/image.png)

where
𝐴
 and
𝐵
 are events and
𝑃
(
𝐵
)
≠
0
.

𝑃
(
𝐴
|
𝐵
)
 is a conditional probability: the probability of event
𝐴
 occurring given that
𝐵
 is true. It is also called the posterior probability of
𝐴
 given
𝐵
.
𝑃
(
𝐵
|
𝐴
)
 is also a conditional probability: the probability of event
𝐵
 occurring given that
𝐴
 is true. It can also be interpreted as the likelihood of
𝐴
 given a fixed
𝐵
 because
𝑃
(
𝐵
|
𝐴
)
=
𝐿
(
𝐴
|
𝐵
)
.
𝑃
(
𝐴
)
 and
𝑃
(
𝐵
)
 are the probabilities of observing
𝐴
 and
𝐵
 respectively without any given conditions; they are known as the prior probability and marginal probability.

In [ ]:
np.zeros((2, 10))

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
import numpy as np
import pandas as pd


class OurNaiveBayesClassifier:
    def __init__(self):
        self.means = None
        self.vars = None
        self.priors = None
        self.classes = None

    def fit(self, X, y) -> None:
        self.classes = np.unique(y) # [0, 1]
        n_samples, n_features = X.shape

        self.means = np.zeros((len(self.classes), n_features))
        self.vars = np.zeros((len(self.classes), n_features))
        self.priors = np.zeros((len(self.classes)))

        for idx, cls in enumerate(self.classes):
            sub_X = X[y==cls] # filtered only specific data features for specific class

            self.means[idx, :] = np.mean(sub_X, axis=0) # NOTE: might be invalid use sub_X.mean(axis=0)
            self.vars[idx, :] = np.var(sub_X, axis=0)

            self.priors[idx] = sub_X.shape[0] / n_samples #sub_X.shape[0] -> number of samples

    def predict(self, X)-> pd.Series:
        log_probs = self.calculate_log_probability(X)
        return pd.Series(self.classes[np.argmax(log_probs, axis=1)])

    def calculate_log_probability(self, X) -> np.ndarray:
        log_probs = np.zeros((X.shape[0], len(self.classes)))

        for idx, cls in enumerate(self.classes):
            prior_log = np.log(self.priors[idx])
            likelihood_log = -0.5 * np.sum(np.log(2.0 * np.pi * self.vars[idx]) + ((X - self.means[idx])**2) / (2.0 * self.vars[idx]), axis=1)
            log_probs[:, idx] = prior_log + likelihood_log
        return log_probs


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
our_model = OurNaiveBayesClassifier()
our_model.fit(X_train, y_train)
predicted_y = our_model.predict(X_test)


<ipython-input-23-8e6b9361e59a>:37: RuntimeWarning: divide by zero encountered in log
  likelihood_log = -0.5 * np.sum(np.log(2.0 * np.pi * self.vars[idx]) + ((X - self.means[idx])**2) / (2.0 * self.vars[idx]), axis=1)


In [ ]:
accuracy_score(y_test, predicted_y)

0.7943519188993483

In [ ]:
from sklearn.naive_bayes import GaussianNB

sklearn_model = GaussianNB()
sklearn_model.fit(X_train, y_train)
sklearn_predicted_y = sklearn_model.predict(X_test)

accuracy_score(y_test, sklearn_predicted_y)

0.8247646632874729